# Requirements and Imports

We import the required libraries

In [1]:
# General imports
import random
import re
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
from google.colab import drive
from google.colab import files
import matplotlib.pyplot as plt
from matplotlib import cm

We mount Google Drive for files needed

In [5]:
# Drive mounting
drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/TESI/TESTS'
if os.path.exists(folder_path):
    print(f"The folder exists at {folder_path}")
else:
    raise FileNotFoundError("The folder could not be found.")
os.chdir(folder_path)
folderpaths = [
    os.path.join(folder_path, 'shell'),
    os.path.join(folder_path, 'shell (1)')
]
print("Resolved folder paths:")
for path in folderpaths:
    print(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
The folder exists at /content/drive/MyDrive/TESI/TESTS
Resolved folder paths:
/content/drive/MyDrive/TESI/TESTS/shell
/content/drive/MyDrive/TESI/TESTS/shell (1)


# Parsing and Loading

In this section we have the functions to parse the output files from the planners and loading the data into pandas data frames used for visualizations of the results in the following sections

We start setting the general parameters

In [ ]:
# Define planners structures

fd_preprocessing = []
fd_searching = []
fd_steps = []
fd_costs = []

symba_2_preprocessing = []
symba_2_searching = []
symba_2_steps = []
symba_2_costs = []



# Function to parse the output file
def parse_output_file(output_file):

    with open(output_file, 'r') as f:
        lines = f.readlines()

    assert lines, "The file is empty or could not be read."

    preprocessing = []
    searching = []
    steps =[]
    costs = []

    for line in lines:

      # Skip blank lines and non-significant lines
      if not re.search(r'(Plan length|Plan cost|Search time|Total time)', line):
        continue

      # Extracting plan length
      plan_length_match = re.search(r'Plan length: (\d+) step\(s\)', line)

      if plan_length_match is not None:
        plan_length = int(plan_length_match.group(1))
        steps.append(plan_length)
        continue

      # Extracting plan cost
      plan_cost_match = re.search(r'Plan cost: (\d+)', line)

      if plan_cost_match is not None:
        plan_cost = int(plan_cost_match.group(1))
        costs.append(plan_cost)
        continue


      # Extracting search time
      search_time_match = re.search(r'Search time: ([0-9.]+)s', line)

      if search_time_match is not None:
        search_time = float(search_time_match.group(1))
        searching.append(search_time)
        continue


      # Extracting total time
      total_time_match = re.search(r'Total time: ([0-9.]+)s', line)

      if total_time_match is not None:
        total_time = float(total_time_match.group(1))
        preprocessing_time = total_time - search_time
        preprocessing.append(preprocessing_time)
        continue

      preprocessing_time = total_time - search_time
      preprocessing.append(preprocessing_time)

    if sum(preprocessing) / len(preprocessing) == 0 and sum(searching) / len(searching) == 0 and round(sum(steps) / len(steps),0) == 0 and round(sum(costs) / len(costs),0) == 0:
      preprocessing = 0
      searching = 0
      steps = 0
      costs = 0
      return preprocessing, searching, steps, costs

    return sum(preprocessing) / len(preprocessing), sum(searching) / len(searching), sum(steps) / len(steps), sum(costs) / len(costs)

def generate_df(folder_path):
    results = []

    # Parse each result trace alignment file
    for folderpath in folderpaths:
      for filename in os.listdir(folderpath):
          # Extract planner type, trace_length, noise_percentage, alphabet_length, constraint modified, and grounding_timen from filename
          if filename.endswith('.txt'):
            parts = filename.split('_')

            assert len(parts) == 7, f"Invalid filename format: {filename} , parts: {parts}"

            # Filename information extraction
            planner_type = parts[0]
            trace_length = parts[1]
            noise_percentage = parts[2]
            alphabet = parts[3]
            constraint_modified = parts[4]
            grounding_time = parts[5].replace('.txt', '')

            # Verify if the execution ended
            if grounding_time != '0':
                # Parse the output file and take for each configuration the averaged results
                preprocessing, searching, steps, costs = parse_output_file(os.path.join(folderpath, filename))
            else: # exceeded the threshold
                preprocessing = '-'
                searching = '-'
                steps = '-'
                costs = '-'
                grounding_time = '-'

            # Store results in a list of dictionaries
            results.append({
                'planner_type': planner_type,
                'trace_length': trace_length,
                'noise_percentage': noise_percentage,
                'alphabet': alphabet,
                'constraint_modified': constraint_modified,
                'grounding_time': grounding_time,
                'fd_preprocessing' if planner_type == "fast-downward" else 'symba_preprocessing': preprocessing,
                'fd_searching' if planner_type == "fast-downward" else 'symba_searching': searching,
                'fd_steps'if planner_type == "fast-downward" else 'symba_steps': steps,
                'fd_costs'if planner_type == "fast-downward" else 'symba_costs': costs,
            })

    # Create a DataFrame from the results
    df = pd.DataFrame(results)

    # Optionally export to csv
    #results_file = f'{planner_type}_{trace_length}_{noise_percentage}%_{alphabet}_{constraint_modified}_{grounding_time}.csv'
    #df.to_csv(results_file)
    #files.download(results_file)

    return df


# Results

In this section we generate the plots coming from the results obtained running the planners

We generate the table showing the overall results obtained with all the planners varying different conditions

In [ ]:
def reshaping_df(df):

    reshaped_data = {
        "Trace Length": [],
        "Noise Percentage": [],
        "Alphabet":[],
        "Constraints": [],
        "Grounding Time": [],
        "Fast-Downward Preprocessing": [],
        "Fast-Downward Searching": [],
        "Fast-Downward Steps": [],
        "Fast-Downward Costs": [],
        "Symba Preprocessing": [],
        "Symba Searching": [],
        "Symba Steps": [],
        "Symba Costs": []
    }

    for index, row in df.iterrows():
        planner_type = row["planner_type"]
        reshaped_data["Trace Length"].append(row["trace_length"])
        reshaped_data["Noise Percentage"].append(row["noise_percentage"])
        reshaped_data["Alphabet"].append(row["alphabet"])
        reshaped_data["Constraints"].append(row["constraint_modified"])
        reshaped_data["Grounding Time"].append(row["grounding_time"])
        reshaped_data["Fast-Downward Preprocessing"].append(row["fd_preprocessing"] if planner_type == "fast-downward" else 0)
        reshaped_data["Fast-Downward Searching"].append(row["fd_searching"] if planner_type == "fast-downward" else 0)
        reshaped_data["Fast-Downward Steps"].append(row["fd_steps"] if planner_type == "fast-downward" else 0)
        reshaped_data["Fast-Downward Costs"].append(row["fd_costs"] if planner_type == "fast-downward" else 0)
        reshaped_data["Symba Preprocessing"].append(row["symba_preprocessing"] if planner_type == "symba2" else 0)
        reshaped_data["Symba Searching"].append(row["symba_searching"] if planner_type == "symba2" else 0)
        reshaped_data["Symba Steps"].append(row["symba_steps"] if planner_type == "symba2" else 0)
        reshaped_data["Symba Costs"].append(row["symba_costs"] if planner_type == "symba2" else 0)


    return pd.DataFrame(reshaped_data)


def generate_multi_indexed_table(df):

    # Melt the DataFrame to long format for easier plotting

    melted_df = df.melt(id_vars=["Trace Length", "Noise Percentage", "Alphabet", "Constraints", "Grounding Time"],
                        value_vars=["Fast-Downward Preprocessing", "Fast-Downward Searching", "Fast-Downward Steps", "Fast-Downward Costs","Symba Preprocessing", "Symba Searching", "Symba Steps", "Symba Costs"],
                        var_name="Metric",
                        value_name="Value")

    # Create a pivot table with Metric as columns
    pivot_table = melted_df.pivot_table(index=["Trace Length", "Noise Percentage", "Alphabet", "Constraints", "Grounding Time"],
                                        columns="Metric",
                                        values="Value",
                                        )


    return pivot_table



# Execution

In this section we execute the overall code

In [ ]:
def main():
    # Loading pipeline for results files
    df = generate_df(folder_path)
    # Reshaping to the desired format
    df = reshaping_df(df)
    # Visualization
    df = generate_multi_indexed_table(df)
    # Export to csv
    df.to_csv('results.csv')
    files.download('results.csv')

    return df

main()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Metric                                                             Fast-Downward Costs  \
Trace Length Noise Percentage Alphabet Constraints Grounding Time                        
10           0%               8        1           12933                           5.0   
                                                   8078                            0.0   
                                       3           19632                           0.0   
                                                   32414                           5.0   
                                       5           19530                           0.0   
...                                                                                ...   
30           30%              8        3           86163                           0.0   
                                       5           156622                         23.0   
                                                   85916                           0.0   
                                       7           100127                          0.0   
                                                   181681                         23.6   

Metric                                                             Fast-Downward Preprocessing  \
Trace Length Noise Percentage Alphabet Constraints Grounding Time                                
10           0%               8        1           12933                              0.131565   
                                                   8078                               0.000000   
                                       3           19632                              0.000000   
                                                   32414                              0.139276   
                                       5           19530                              0.000000   
...                                                                                        ...   
30           30%              8        3           86163                              0.000000   
                                       5           156622                             0.147705   
                                                   85916                              0.000000   
                                       7           100127                             0.000000   
                                                   181681                             0.150133   

Metric                                                             Fast-Downward Searching  \
Trace Length Noise Percentage Alphabet Constraints Grounding Time                            
10           0%               8        1           12933                          0.010006   
                                                   8078                           0.000000   
                                       3           19632                          0.000000   
                                                   32414                          0.011512   
                                       5           19530                          0.000000   
...                                                                                    ...   
30           30%              8        3           86163                          0.000000   
                                       5           156622                         0.034339   
                                                   85916                          0.000000   
                                       7           100127                         0.000000   
                                                   181681                         0.102491   

Metric                                                             Fast-Downward Steps  \
Trace Length Noise Percentage Alphabet Constraints Grounding Time                        
10           0%               8        1           12933                         15.00   
                                                   8078      